# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [11]:

# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'
movie_id_list = []
resp_b = BeautifulSoup(resp.text)
reg =re.compile("\d*")
movie_id_list = [{'movie_id':m["value"],'moive_name':m['data-name']} for  m in resp_b.find("select",id="sbox_mid",attrs={'name':'movie_id'}).findChildren("option",attrs={'data-name':re.compile(".*")}) if  re.search(reg,m['value'])]
print(movie_id_list)

[{'movie_id': '11428', 'moive_name': '極速引爆'}, {'movie_id': '11424', 'moive_name': '手捲煙'}, {'movie_id': '11397', 'moive_name': '群青戰記'}, {'movie_id': '11388', 'moive_name': '宇宙特攻隊'}, {'movie_id': '11376', 'moive_name': '厲陰宅3：是惡魔逼我的'}, {'movie_id': '11391', 'moive_name': '花束般的戀愛'}, {'movie_id': '11375', 'moive_name': '貝加爾湖隱居札記：在這喧囂的世界，一個人到西伯利亞森林住半年'}, {'movie_id': '11342', 'moive_name': '怪物奇兵 全新世代'}, {'movie_id': '11426', 'moive_name': '韓國校園恐怖怪談：點名沒到的學生'}, {'movie_id': '11332', 'moive_name': '燕尾蝶：數位經典版'}, {'movie_id': '11329', 'moive_name': '轉彎之後'}, {'movie_id': '11300', 'moive_name': '野蠻糾碴隊'}, {'movie_id': '11201', 'moive_name': '失控的審判'}, {'movie_id': '10397', 'moive_name': '黑寡婦'}, {'movie_id': '11345', 'moive_name': '迴憶'}, {'movie_id': '11340', 'moive_name': '玩命鈔劫'}, {'movie_id': '11311', 'moive_name': '死亡漩渦：奪魂鋸新遊戲'}, {'movie_id': '11206', 'moive_name': '名偵探柯南：緋色的彈丸'}, {'movie_id': '11370', 'moive_name': '鬼護士'}, {'movie_id': '11355', 'moive_name': '如果雨之後'}, {'movie_id': '11352', 'moive_

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [12]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 11376

In [13]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
parameter = {'movie_id':movie_id}
header = {
'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb'
}
resp_areas = requests.get(url,params=parameter,headers=header)
print([{'area_name':d['title'],'area_id':d['area_id']} for d in resp_areas.json()])

[{'area_name': '台北市', 'area_id': 28}, {'area_name': '新竹', 'area_id': 20}, {'area_name': '苗栗', 'area_id': 15}, {'area_name': '台中', 'area_id': 2}, {'area_name': '台南', 'area_id': 10}, {'area_name': '高雄', 'area_id': 17}, {'area_name': '澎湖', 'area_id': 23}]


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 28

In [14]:
# 向網站發送請求
params = {'movie_id':movie_id,'area_id':area_id}
url = f"https://movies.yahoo.com.tw/movietime_result.html"
resp = requests.get(url,params=params)
resp_b = BeautifulSoup(resp.text)
print([{'month':d.find('p',class_='month').text,'day':d.find('h3',class_='day').text,'week':d.find('p',class_='week').text} for d in resp_b.find('ul',class_='date_timetable_picker').find_all('li')])
# 列印播放日期

[{'month': '七月', 'day': '22', 'week': '今天'}, {'month': '七月', 'day': '23', 'week': '明天'}, {'month': '七月', 'day': '24', 'week': '禮拜六'}, {'month': '七月', 'day': '25', 'week': '禮拜天'}, {'month': '七月', 'day': '26', 'week': '禮拜一'}]


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [8]:
# 選定要觀看的日期
date = "2021-07-25"

In [9]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [17]:
# 向網站發送請求，獲取上映的電影院及時間資訊
params = {'movie_id':movie_id,'area_id':area_id,'date':date}
url = f"https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
resp = requests.get(url,params=params)
resp_b = BeautifulSoup(resp.json()['view'])

print(resp.json()['view'])  # 若有需要，列印出json原始碼

<div class="pc-movie-schedule-form">
    <div class="area_timebox">
   <div class="area_title">台北市</div>
      <ul id="theater_id_32" class="area_time _c jq_area_time" data-theater_name="台北美麗華大直影城" data-theater_url="http://www.miramarcinemas.tw/" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=32" >
       <li class="adds">
           <a href="https://movies.yahoo.com.tw/theater_result.html/id=32">台北美麗華大直影城</a>
           <span>02-85022208</span>
       </li>
                         <li class="taps">
                <span class="tapR">數位</span>                           </li>
           <li class="time _c">
               <div class="input_picker jq_input_picker">
                                                                   <input type="radio" name="schedule_list" id="75540873" class="gabtn"
                        value="2021-07-25 09:10:00"  data-movie="12725c7aa8084d638bd0af96603a0b6d"
                        data-cinema="40288eb0050f31210ada0f55676

In [18]:
for h in html:
    print('===')
    print(h.find('input'))
    print('---')
    print(h.find('input')['value'])

===
<input class="gabtn" data-cinema="40288eb0050f31210ada0f55676b1bbc" data-date="1627142400" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '厲陰宅3：是惡魔逼我的']" data-movie="12725c7aa8084d638bd0af96603a0b6d" data-movie_date="07.25" data-movie_time="09:10" data-movie_title="厲陰宅3：是惡魔逼我的" data-movie_type="數位" data-session="a6836bbb4a6849ff9ae44d4d22ae6f08" data-time="1627175400" id="75540873" name="schedule_list" type="radio" value="2021-07-25 09:10:00"/>
---
2021-07-25 09:10:00
===
<input class="gabtn" data-cinema="cf2b5aa8132f11e6b767a1b123456789" data-date="1627142400" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '厲陰宅3：是惡魔逼我的']" data-movie="34ea7f49aeb342d7965ebf535901b3ba" data-movie_date="07.25" data-movie_time="14:40" data-movie_title="厲陰宅3：是惡魔逼我的" data-movie_type="數位" data-session="7195cb008a574b13bb6950c96ee5d961" data-time="1627195200" id="75542683" name="schedule_list" type="radio" value="2021-07-25 14:40:00"/>
---
2021-07-25 14:40:00
===
<input class="gabtn" dat

In [20]:

for h in html:
    theater = h['data-theater_name']
    time = h.find('input')['value']
    t = h.find('li', attrs={'class':"taps"})
    print("Time:", time, "Type:", t.find('span').text, "Theater:", theater)

Time: 2021-07-25 09:10:00 Type: 數位 Theater: 台北美麗華大直影城
Time: 2021-07-25 14:40:00 Type: 數位 Theater: 台北新光影城
Time: 2021-07-25 13:10:00 Type: 數位 Theater: 喜滿客絕色影城
Time: 2021-07-25 10:10:00 Type: 數位 Theater: 哈拉影城
Time: 2021-07-25 12:30:00 Type: 數位 Theater: 梅花數位影院
Time: 2021-07-25 12:30:00 Type: 數位 Theater: 台北新民生戲院
Time: 2021-07-25 11:00:00 Type: 數位 Theater: 喜樂時代影城南港店
